In [73]:
import pandas as pd
import re

import string
import spacy
from spacy import displacy

In [74]:
NLP = spacy.load("en_core_web_sm")

In [75]:
nike_data = pd.read_csv('nike_data.csv', index_col=0)
nike_data.dropna(inplace = True)

In [76]:
nike_data.iloc[1]['caption']

'Meet @azusa25nigo, the founder of @skate_girls_snap. 🛹\n\nSkate Girls Snap is a project that celebrates the diversity within Tokyo’s growing women’s skate scene. Through this project, Azusa is creating a more inclusive skate culture and inspiring the \nnext generation of female skaters to be themselves.\n\nLearn more about Skate Girls Snap and Azusa’s skate crew. Link in bio.\n\n🎥: @nobu_arakawa @callumhasegawa'

In [77]:
nike_data.shape

(623, 5)

In [78]:
nike_data.columns

Index(['img_url', 'caption', 'n_likes_1000', 'n_comments', 'age'], dtype='object')

In [79]:
nike_data.head()

,img_url,caption,n_likes_1000,n_comments,age
0,https://scontent-waw1-1.cdninstagram.com/v/t51...,“100% of myself is nothing compared to 1% of t...,218k,9590,1 day ago
1,https://scontent-waw1-1.cdninstagram.com/v/t51...,"Meet @azusa25nigo, the founder of @skate_girls...",77k,62680,2 days ago
2,https://scontent-waw1-1.cdninstagram.com/v/t51...,It takes courage to take the first step 🏃. Jus...,232k,46169,2 days ago
3,https://scontent-waw1-1.cdninstagram.com/v/t51...,“The climate crisis is affecting my sport and ...,158k,86967,6 days ago
4,https://scontent-waw1-1.cdninstagram.com/v/t51...,“People like to tell us what we can and can’t ...,251k,67604,1 week ago


In [80]:
def wrangle(text):
    text = text.replace('“', '"').replace('”','"').replace('’', "'")
    printable = set(string.printable)
    text = ''.join(filter(lambda x: x in printable, text))
    return text

In [176]:
def firstLineQuote(text):
    
    pattern_qp = r'\"(.*?)\"[@\-\w\s]+'
    pattern_q = r'\"(.*?)\"'
    r_qp = re.fullmatch(pattern_qp,text)
    r_q = re.fullmatch(pattern_q, text)
    if r_qp != None:
        return 'personal_quote'
    if r_q != None:
        return 'quote'
    return ''

In [109]:
def generateNER(text):
    doc = NLP(text)
    
    ners = []
    for word in doc.ents:
        if word.label_.lower() in ['event', 'fac', 'gpe', 'law', 'loc', 'money', 'norp', 'org', 'person', 'product', 'work_of_art']:
            ners.append(word.label_.lower())
    
    return ','.join(ners)
    

In [172]:
def analyzeText(text):
    
    print("----------- Actual Text ------------")
    print(text)
    doc = NLP(text)
    
#     print("----------- Spacy render -----------")
#     displacy.render(doc,style="ent",jupyter=True)
    
#     print("----------- Spacy NERs -------------")
#     for word in doc.ents:
#         print(word.text,word.label_)
    
#     print("----------- POS tag ----------------")
#     # Token and Tag
#     for token in doc:
#         print(token, token.pos_)        

In [ ]:
def 

In [82]:
nike_data['caption_cleaned'] = nike_data['caption'].apply(lambda x : wrangle(x))

In [107]:
nike_data['NER'] = nike_data['caption_cleaned'].apply(lambda x : generateNER(x))

In [177]:
nike_data['firstLineQuote'] = nike_data['caption_cleaned'].apply(lambda x : firstLineQuote(x.strip().split('\n')[0].strip())) 

In [181]:
nike_data.head()

,img_url,caption,n_likes_1000,n_comments,age,caption_cleaned,NER,firstLineQuote
0,https://scontent-waw1-1.cdninstagram.com/v/t51...,“100% of myself is nothing compared to 1% of t...,218k,9590,1 day ago,"""100% of myself is nothing compared to 1% of t...",person,personal_quote
1,https://scontent-waw1-1.cdninstagram.com/v/t51...,"Meet @azusa25nigo, the founder of @skate_girls...",77k,62680,2 days ago,"Meet @azusa25nigo, the founder of @skate_girls...","person,gpe,org,norp,work_of_art,org",
2,https://scontent-waw1-1.cdninstagram.com/v/t51...,It takes courage to take the first step 🏃. Jus...,232k,46169,2 days ago,It takes courage to take the first step . Just...,"org,gpe,org,gpe,norp,work_of_art,org",
3,https://scontent-waw1-1.cdninstagram.com/v/t51...,“The climate crisis is affecting my sport and ...,158k,86967,6 days ago,"""The climate crisis is affecting my sport and ...",person,quote
4,https://scontent-waw1-1.cdninstagram.com/v/t51...,“People like to tell us what we can and can’t ...,251k,67604,1 week ago,"""People like to tell us what we can and can't ...","org,org,org,org",personal_quote
